# BDH Curriculum Training Notebook

This notebook runs the full curriculum learning pipeline for BDH:

**5-Phase Curriculum:**
1. **Primitives** - Simple, heavily commented single functions
2. **Composition** - Loops, recursion, control flow
3. **Algorithms** - Classic CS algorithms (sorting, searching, DP)
4. **Systems** - OOP, design patterns, utilities
5. **Language** - Natural language (WikiText-2)

**Hypothesis:** Training on progressively complex code before natural language
will build better generalization circuits than training on language directly.

---

## 0. Setup Environment

In [ ]:
# Check if running on Colab
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    # Mount Google Drive for persistent storage
    from google.colab import drive
    drive.mount('/content/drive')
    
    # Clone repo if not exists
    !git clone https://github.com/newsbubbles/bdh.git 2>/dev/null || echo 'Repo exists'
    %cd bdh
    
    # Install dependencies
    !pip install -q torch datasets tqdm
else:
    print('Running locally')
    # Ensure we're in project root
    import os
    if os.path.basename(os.getcwd()) == 'notebooks':
        os.chdir('..')

In [ ]:
# Verify GPU
import torch

if torch.cuda.is_available():
    device = 'cuda'
    print(f'✅ GPU: {torch.cuda.get_device_name(0)}')
    print(f'   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB')
else:
    device = 'cpu'
    print('⚠️ No GPU detected, training will be slow')

## 1. Generate/Download Curriculum Data

This section generates all 5 phases of curriculum data.
- Phases 1-4: Synthetically generated Python code
- Phase 5: WikiText-2 natural language

In [ ]:
import json
import random
from pathlib import Path
from datetime import datetime

# Create data directories
DATA_DIR = Path('data/curriculum')
DATA_DIR.mkdir(parents=True, exist_ok=True)

for phase in ['phase1_primitives', 'phase2_composition', 'phase3_algorithms', 
              'phase4_systems', 'phase5_language']:
    (DATA_DIR / phase).mkdir(exist_ok=True)

print('Data directories created:')!ls -la data/curriculum/

### Phase 1: Primitives
Simple single-purpose functions with heavy comments.

In [ ]:
# Phase 1: Primitives Generator
# Simple, heavily commented functions

PHASE1_TEMPLATES = {
    'arithmetic': [
        '''def add(a, b):
    """Add two numbers together.
    
    Args:
        a: First number
        b: Second number
    
    Returns:
        The sum of a and b
    """
    # Add the two numbers
    result = a + b
    return result''',
        '''def subtract(a, b):
    """Subtract b from a.
    
    Args:
        a: Number to subtract from
        b: Number to subtract
    
    Returns:
        The difference a - b
    """
    # Calculate the difference
    result = a - b
    return result''',
        '''def multiply(a, b):
    """Multiply two numbers.
    
    Args:
        a: First factor
        b: Second factor
    
    Returns:
        The product of a and b
    """
    # Multiply the numbers
    result = a * b
    return result''',
        '''def divide(a, b):
    """Divide a by b.
    
    Args:
        a: Dividend
        b: Divisor (must not be zero)
    
    Returns:
        The quotient a / b
    """
    # Perform division
    result = a / b
    return result''',
        '''def square(x):
    """Calculate the square of a number.
    
    Args:
        x: Number to square
    
    Returns:
        x squared (x * x)
    """
    # Square the number
    return x * x''',
    ],
    'comparison': [
        '''def is_positive(n):
    """Check if a number is positive.
    
    Args:
        n: Number to check
    
    Returns:
        True if n > 0, False otherwise
    """
    # Check if greater than zero
    return n > 0''',
        '''def is_even(n):
    """Check if a number is even.
    
    Args:
        n: Integer to check
    
    Returns:
        True if n is divisible by 2
    """
    # Even numbers have no remainder when divided by 2
    return n % 2 == 0''',
        '''def max_of_two(a, b):
    """Return the larger of two numbers.
    
    Args:
        a: First number
        b: Second number
    
    Returns:
        The larger value
    """
    # Compare and return larger
    if a > b:
        return a
    return b''',
    ],
    'string': [
        '''def get_length(s):
    """Get the length of a string.
    
    Args:
        s: Input string
    
    Returns:
        Number of characters in s
    """
    # Use len() to count characters
    return len(s)''',
        '''def to_upper(s):
    """Convert string to uppercase.
    
    Args:
        s: Input string
    
    Returns:
        String in all uppercase
    """
    # Convert to uppercase
    return s.upper()''',
    ],
}

def generate_phase1_variations(templates, num_variations=200):
    """Generate variations of templates with different variable names."""
    examples = []
    var_names = ['x', 'y', 'n', 'm', 'val', 'num', 'value', 'input_val', 'data']
    
    for category, funcs in templates.items():
        for func in funcs:
            # Add original
            examples.append({
                'code': func,
                'category': category,
                'difficulty': 0.1,
                'phase': 1
            })
            
            # Add variations
            for _ in range(num_variations // len(funcs)):
                varied = func
                # Simple variable substitution
                old_var = random.choice(['a', 'b', 'x', 'n'])
                new_var = random.choice(var_names)
                if old_var in varied and new_var not in varied:
                    varied = varied.replace(old_var, new_var)
                examples.append({
                    'code': varied,
                    'category': category,
                    'difficulty': 0.1,
                    'phase': 1
                })
    
    return examples

# Generate Phase 1
phase1_dir = DATA_DIR / 'phase1_primitives'
phase1_file = phase1_dir / 'phase1_primitives.jsonl'

if not phase1_file.exists():
    print('Generating Phase 1: Primitives...')
    examples = generate_phase1_variations(PHASE1_TEMPLATES, num_variations=500)
    random.shuffle(examples)
    
    with open(phase1_file, 'w') as f:
        for ex in examples:
            f.write(json.dumps(ex) + '\n')
    
    # Stats
    tokens = sum(len(ex['code']) // 4 for ex in examples)
    stats = {'total': len(examples), 'tokens': tokens}
    with open(phase1_dir / 'phase1_stats.json', 'w') as f:
        json.dump(stats, f)
    
    print(f'  Generated {len(examples)} examples, ~{tokens:,} tokens')
else:
    with open(phase1_dir / 'phase1_stats.json') as f:
        stats = json.load(f)
    print(f'Phase 1 exists: {stats}')

### Phase 2: Composition
Functions with loops, recursion, and control flow.

In [ ]:
# Phase 2: Composition Generator
# Loops, recursion, control flow

PHASE2_TEMPLATES = {
    'loops': [
        '''def sum_list(numbers):
    """Calculate the sum of all numbers in a list.
    
    Args:
        numbers: List of numbers to sum
    
    Returns:
        Total sum of all numbers
    """
    # Initialize sum to zero
    total = 0
    
    # Add each number to the total
    for num in numbers:
        total = total + num
    
    return total''',
        '''def find_max(numbers):
    """Find the maximum value in a list.
    
    Args:
        numbers: List of numbers
    
    Returns:
        The largest number in the list
    """
    # Start with first element
    max_val = numbers[0]
    
    # Check each number
    for num in numbers:
        if num > max_val:
            max_val = num
    
    return max_val''',
        '''def count_occurrences(items, target):
    """Count how many times target appears in items.
    
    Args:
        items: List to search
        target: Value to count
    
    Returns:
        Number of occurrences
    """
    # Initialize counter
    count = 0
    
    # Check each item
    for item in items:
        if item == target:
            count = count + 1
    
    return count''',
    ],
    'recursion': [
        '''def factorial(n):
    """Calculate n factorial (n!).
    
    Args:
        n: Non-negative integer
    
    Returns:
        n! = n * (n-1) * ... * 1
    """
    # Base case: 0! = 1
    if n == 0:
        return 1
    
    # Recursive case: n! = n * (n-1)!
    return n * factorial(n - 1)''',
        '''def fibonacci(n):
    """Calculate the nth Fibonacci number.
    
    Args:
        n: Index in Fibonacci sequence
    
    Returns:
        The nth Fibonacci number
    """
    # Base cases
    if n <= 1:
        return n
    
    # Recursive case: F(n) = F(n-1) + F(n-2)
    return fibonacci(n - 1) + fibonacci(n - 2)''',
    ],
    'control_flow': [
        '''def grade_score(score):
    """Convert numeric score to letter grade.
    
    Args:
        score: Numeric score (0-100)
    
    Returns:
        Letter grade (A, B, C, D, or F)
    """
    # Check score ranges
    if score >= 90:
        return "A"
    elif score >= 80:
        return "B"
    elif score >= 70:
        return "C"
    elif score >= 60:
        return "D"
    else:
        return "F"''',
    ],
}

# Generate Phase 2
phase2_dir = DATA_DIR / 'phase2_composition'
phase2_file = phase2_dir / 'phase2_composition.jsonl'

if not phase2_file.exists():
    print('Generating Phase 2: Composition...')
    examples = []
    
    for category, funcs in PHASE2_TEMPLATES.items():
        for func in funcs:
            # Add multiple copies with slight variations
            for i in range(100):
                examples.append({
                    'code': func,
                    'category': category,
                    'difficulty': 0.25,
                    'phase': 2
                })
    
    random.shuffle(examples)
    
    with open(phase2_file, 'w') as f:
        for ex in examples:
            f.write(json.dumps(ex) + '\n')
    
    tokens = sum(len(ex['code']) // 4 for ex in examples)
    stats = {'total': len(examples), 'tokens': tokens}
    with open(phase2_dir / 'phase2_stats.json', 'w') as f:
        json.dump(stats, f)
    
    print(f'  Generated {len(examples)} examples, ~{tokens:,} tokens')
else:
    with open(phase2_dir / 'phase2_stats.json') as f:
        stats = json.load(f)
    print(f'Phase 2 exists: {stats}')

### Phase 3: Algorithms
Classic CS algorithms - sorting, searching, data structures.

In [ ]:
# Phase 3: Algorithms

PHASE3_TEMPLATES = {
    'sorting': [
        '''def bubble_sort(arr):
    """Sort array using bubble sort algorithm.
    
    Bubble sort repeatedly steps through the list, compares adjacent
    elements and swaps them if they are in the wrong order.
    
    Time complexity: O(n^2)
    Space complexity: O(1)
    
    Args:
        arr: List of comparable elements
    
    Returns:
        Sorted list in ascending order
    """
    n = len(arr)
    
    # Traverse through all elements
    for i in range(n):
        # Last i elements are already in place
        for j in range(0, n - i - 1):
            # Swap if current element is greater than next
            if arr[j] > arr[j + 1]:
                arr[j], arr[j + 1] = arr[j + 1], arr[j]
    
    return arr''',
        '''def insertion_sort(arr):
    """Sort array using insertion sort algorithm.
    
    Builds sorted array one element at a time by inserting
    each element into its correct position.
    
    Time complexity: O(n^2)
    Space complexity: O(1)
    
    Args:
        arr: List of comparable elements
    
    Returns:
        Sorted list in ascending order
    """
    # Start from second element
    for i in range(1, len(arr)):
        key = arr[i]
        j = i - 1
        
        # Move elements greater than key ahead
        while j >= 0 and arr[j] > key:
            arr[j + 1] = arr[j]
            j = j - 1
        
        # Insert key at correct position
        arr[j + 1] = key
    
    return arr''',
    ],
    'searching': [
        '''def binary_search(arr, target):
    """Search for target in sorted array using binary search.
    
    Binary search divides the search interval in half repeatedly.
    
    Time complexity: O(log n)
    Space complexity: O(1)
    
    Args:
        arr: Sorted list of elements
        target: Element to find
    
    Returns:
        Index of target if found, -1 otherwise
    """
    left = 0
    right = len(arr) - 1
    
    while left <= right:
        # Find middle index
        mid = (left + right) // 2
        
        if arr[mid] == target:
            return mid
        elif arr[mid] < target:
            left = mid + 1
        else:
            right = mid - 1
    
    return -1''',
    ],
    'data_structures': [
        '''class Stack:
    """A simple stack implementation using a list.
    
    Stack follows Last-In-First-Out (LIFO) principle.
    """
    
    def __init__(self):
        """Initialize an empty stack."""
        self.items = []
    
    def push(self, item):
        """Add item to top of stack."""
        self.items.append(item)
    
    def pop(self):
        """Remove and return top item."""
        if not self.is_empty():
            return self.items.pop()
        return None
    
    def peek(self):
        """Return top item without removing."""
        if not self.is_empty():
            return self.items[-1]
        return None
    
    def is_empty(self):
        """Check if stack is empty."""
        return len(self.items) == 0''',
    ],
}

# Generate Phase 3
phase3_dir = DATA_DIR / 'phase3_algorithms'
phase3_file = phase3_dir / 'phase3_algorithms.jsonl'

if not phase3_file.exists():
    print('Generating Phase 3: Algorithms...')
    examples = []
    
    for category, funcs in PHASE3_TEMPLATES.items():
        for func in funcs:
            for i in range(50):
                examples.append({
                    'code': func,
                    'category': category,
                    'difficulty': 0.5,
                    'phase': 3
                })
    
    random.shuffle(examples)
    
    with open(phase3_file, 'w') as f:
        for ex in examples:
            f.write(json.dumps(ex) + '\n')
    
    tokens = sum(len(ex['code']) // 4 for ex in examples)
    stats = {'total': len(examples), 'tokens': tokens}
    with open(phase3_dir / 'phase3_stats.json', 'w') as f:
        json.dump(stats, f)
    
    print(f'  Generated {len(examples)} examples, ~{tokens:,} tokens')
else:
    with open(phase3_dir / 'phase3_stats.json') as f:
        stats = json.load(f)
    print(f'Phase 3 exists: {stats}')

### Phase 4: Systems
OOP, design patterns, multi-function modules.

In [ ]:
# Phase 4: Systems

PHASE4_TEMPLATES = [
    '''class BankAccount:
    """A simple bank account with deposit and withdraw operations.
    
    Attributes:
        owner: Name of account holder
        balance: Current account balance
    """
    
    def __init__(self, owner, initial_balance=0):
        """Create a new bank account.
        
        Args:
            owner: Name of account holder
            initial_balance: Starting balance (default 0)
        """
        self.owner = owner
        self.balance = initial_balance
    
    def deposit(self, amount):
        """Add money to the account.
        
        Args:
            amount: Amount to deposit (must be positive)
        
        Returns:
            New balance after deposit
        """
        if amount > 0:
            self.balance += amount
        return self.balance
    
    def withdraw(self, amount):
        """Remove money from the account.
        
        Args:
            amount: Amount to withdraw
        
        Returns:
            New balance, or None if insufficient funds
        """
        if amount <= self.balance:
            self.balance -= amount
            return self.balance
        return None
    
    def get_balance(self):
        """Return current balance."""
        return self.balance''',
    '''class Logger:
    """A simple logging utility.
    
    Supports different log levels: DEBUG, INFO, WARNING, ERROR.
    """
    
    DEBUG = 0
    INFO = 1
    WARNING = 2
    ERROR = 3
    
    def __init__(self, name, level=1):
        """Create a new logger.
        
        Args:
            name: Logger name (usually module name)
            level: Minimum log level to output
        """
        self.name = name
        self.level = level
        self.messages = []
    
    def log(self, level, message):
        """Log a message at the specified level."""
        if level >= self.level:
            entry = f"[{self.name}] {message}"
            self.messages.append(entry)
            print(entry)
    
    def debug(self, message):
        """Log a debug message."""
        self.log(self.DEBUG, message)
    
    def info(self, message):
        """Log an info message."""
        self.log(self.INFO, message)
    
    def warning(self, message):
        """Log a warning message."""
        self.log(self.WARNING, message)
    
    def error(self, message):
        """Log an error message."""
        self.log(self.ERROR, message)''',
]

# Generate Phase 4
phase4_dir = DATA_DIR / 'phase4_systems'
phase4_file = phase4_dir / 'phase4_systems.jsonl'

if not phase4_file.exists():
    print('Generating Phase 4: Systems...')
    examples = []
    
    for func in PHASE4_TEMPLATES:
        for i in range(50):
            examples.append({
                'code': func,
                'category': 'oop',
                'difficulty': 0.7,
                'phase': 4
            })
    
    random.shuffle(examples)
    
    with open(phase4_file, 'w') as f:
        for ex in examples:
            f.write(json.dumps(ex) + '\n')
    
    tokens = sum(len(ex['code']) // 4 for ex in examples)
    stats = {'total': len(examples), 'tokens': tokens}
    with open(phase4_dir / 'phase4_stats.json', 'w') as f:
        json.dump(stats, f)
    
    print(f'  Generated {len(examples)} examples, ~{tokens:,} tokens')
else:
    with open(phase4_dir / 'phase4_stats.json') as f:
        stats = json.load(f)
    print(f'Phase 4 exists: {stats}')

### Phase 5: Natural Language (WikiText-2)
Download and prepare WikiText-2 for language modeling.

In [ ]:
# Phase 5: WikiText-2
from datasets import load_dataset

phase5_dir = DATA_DIR / 'phase5_language'
phase5_train = phase5_dir / 'phase5_train.jsonl'

if not phase5_train.exists():
    print('Downloading WikiText-2...')
    dataset = load_dataset('wikitext', 'wikitext-2-raw-v1')
    
    total_tokens = 0
    total_examples = {}
    
    for split in ['train', 'validation', 'test']:
        out_name = 'valid' if split == 'validation' else split
        out_file = phase5_dir / f'phase5_{out_name}.jsonl'
        
        count = 0
        with open(out_file, 'w') as f:
            for item in dataset[split]:
                text = item['text'].strip()
                if len(text) > 50:  # Filter very short
                    f.write(json.dumps({
                        'text': text,
                        'phase': 5,
                        'difficulty': 1.0
                    }) + '\n')
                    count += 1
                    total_tokens += len(text) // 4
        
        total_examples[out_name] = count
        print(f'  {split}: {count} examples')
    
    stats = {
        'total': sum(total_examples.values()),
        'by_split': total_examples,
        'tokens': total_tokens
    }
    with open(phase5_dir / 'phase5_stats.json', 'w') as f:
        json.dump(stats, f)
    
    print(f'  Total: {stats["total"]} examples, ~{total_tokens:,} tokens')
else:
    with open(phase5_dir / 'phase5_stats.json') as f:
        stats = json.load(f)
    print(f'Phase 5 exists: {stats}')

### Verify All Data

In [ ]:
# Verify curriculum data
print('\n' + '='*60)
print('CURRICULUM DATA SUMMARY')
print('='*60 + '\n')

total_examples = 0
total_tokens = 0

phases = [
    ('Phase 1: Primitives', 'phase1_primitives', 'phase1_primitives.jsonl'),
    ('Phase 2: Composition', 'phase2_composition', 'phase2_composition.jsonl'),
    ('Phase 3: Algorithms', 'phase3_algorithms', 'phase3_algorithms.jsonl'),
    ('Phase 4: Systems', 'phase4_systems', 'phase4_systems.jsonl'),
    ('Phase 5: Language', 'phase5_language', 'phase5_train.jsonl'),
]

for name, dir_name, file_name in phases:
    phase_dir = DATA_DIR / dir_name
    jsonl_file = phase_dir / file_name
    stats_file = phase_dir / f'{dir_name.split("_")[0]}_stats.json'
    
    if jsonl_file.exists():
        # Count lines
        with open(jsonl_file) as f:
            count = sum(1 for _ in f)
        
        # Load stats
        if stats_file.exists():
            with open(stats_file) as f:
                stats = json.load(f)
            tokens = stats.get('tokens', 0)
        else:
            tokens = 0
        
        print(f'✅ {name}')
        print(f'   Examples: {count:,}')
        print(f'   Tokens: {tokens:,}')
        total_examples += count
        total_tokens += tokens
    else:
        print(f'❌ {name} - NOT FOUND')
    print()

print('='*60)
print(f'TOTAL: {total_examples:,} examples, ~{total_tokens:,} tokens')
print('='*60)

---
## 2. Model & Training Setup

In [ ]:
# Import BDH model
import sys
sys.path.insert(0, '.')

from bdh import BDH, BDHConfig

# Model configuration
config = BDHConfig(
    n_layer=6,
    n_head=8,
    n_embd=256,
    dropout=0.1,
    vocab_size=256,  # Byte-level
    mlp_internal_dim_multiplier=32,
)

model = BDH(config)
model.to(device)

# Count parameters
n_params = sum(p.numel() for p in model.parameters())
print(f'Model: {config.n_layer}L, {config.n_head}H, {config.n_embd}D')
print(f'Parameters: {n_params:,} ({n_params/1e6:.2f}M)')

In [ ]:
# Dataset class
from torch.utils.data import Dataset, DataLoader

class CurriculumDataset(Dataset):
    """Dataset for curriculum training."""
    
    def __init__(self, jsonl_paths, block_size=256, max_examples=None):
        self.block_size = block_size
        self.examples = []
        
        for path in jsonl_paths:
            if not Path(path).exists():
                continue
            with open(path) as f:
                for line in f:
                    data = json.loads(line)
                    text = data.get('code') or data.get('text', '')
                    if text:
                        self.examples.append(text)
                        if max_examples and len(self.examples) >= max_examples:
                            break
            if max_examples and len(self.examples) >= max_examples:
                break
        
        print(f'Loaded {len(self.examples)} examples')
    
    def __len__(self):
        return len(self.examples)
    
    def __getitem__(self, idx):
        text = self.examples[idx]
        tokens = list(text.encode('utf-8'))
        
        # Pad or crop
        if len(tokens) < self.block_size + 1:
            tokens = tokens + [0] * (self.block_size + 1 - len(tokens))
        else:
            start = random.randint(0, len(tokens) - self.block_size - 1)
            tokens = tokens[start:start + self.block_size + 1]
        
        x = torch.tensor(tokens[:-1], dtype=torch.long)
        y = torch.tensor(tokens[1:], dtype=torch.long)
        return x, y

In [ ]:
# Training configuration
import math
from dataclasses import dataclass

@dataclass
class TrainConfig:
    batch_size: int = 16
    block_size: int = 256
    gradient_accumulation_steps: int = 8
    learning_rate: float = 3e-4
    min_lr: float = 1e-5
    warmup_iters: int = 200
    max_iters: int = 5000
    eval_interval: int = 250
    eval_iters: int = 50
    log_interval: int = 50
    grad_clip: float = 1.0
    weight_decay: float = 0.1

train_config = TrainConfig()

# Optimizer
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=train_config.learning_rate,
    weight_decay=train_config.weight_decay,
    betas=(0.9, 0.95)
)

def get_lr(it, phase_lr):
    """Cosine learning rate schedule with warmup."""
    if it < train_config.warmup_iters:
        return phase_lr * (it + 1) / train_config.warmup_iters
    if it > train_config.max_iters:
        return train_config.min_lr
    decay_ratio = (it - train_config.warmup_iters) / (train_config.max_iters - train_config.warmup_iters)
    coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio))
    return train_config.min_lr + coeff * (phase_lr - train_config.min_lr)

print('Training config ready')

---
## 3. Curriculum Training Loop

Train through all 5 phases sequentially.

In [ ]:
# Phase configurations
PHASES = [
    {
        'name': 'Phase 1: Primitives',
        'files': [DATA_DIR / 'phase1_primitives/phase1_primitives.jsonl'],
        'lr': 3e-4,
        'epochs': 5,
        'target_loss': 1.5,
    },
    {
        'name': 'Phase 2: Composition',
        'files': [DATA_DIR / 'phase2_composition/phase2_composition.jsonl'],
        'lr': 2e-4,
        'epochs': 5,
        'target_loss': 1.4,
    },
    {
        'name': 'Phase 3: Algorithms',
        'files': [DATA_DIR / 'phase3_algorithms/phase3_algorithms.jsonl'],
        'lr': 1.5e-4,
        'epochs': 5,
        'target_loss': 1.3,
    },
    {
        'name': 'Phase 4: Systems',
        'files': [DATA_DIR / 'phase4_systems/phase4_systems.jsonl'],
        'lr': 1e-4,
        'epochs': 5,
        'target_loss': 1.2,
    },
    {
        'name': 'Phase 5: Language',
        'files': [DATA_DIR / 'phase5_language/phase5_train.jsonl'],
        'val_files': [DATA_DIR / 'phase5_language/phase5_valid.jsonl'],
        'lr': 1e-4,
        'epochs': 10,
        'target_loss': 1.1,
    },
]

print('Phase configurations loaded')
for i, phase in enumerate(PHASES, 1):
    print(f"  {i}. {phase['name']} (lr={phase['lr']}, epochs={phase['epochs']})")

In [ ]:
# Training loop
import time
from tqdm.auto import tqdm

# Create checkpoint directory
CHECKPOINT_DIR = Path('checkpoints_curriculum')
CHECKPOINT_DIR.mkdir(exist_ok=True)

# Training history
history = {
    'train_loss': [],
    'val_loss': [],
    'phase': [],
    'iteration': [],
}

global_iter = 0
best_val_loss = float('inf')

@torch.no_grad()
def evaluate(model, loader, max_batches=50):
    """Evaluate model on dataset."""
    model.eval()
    losses = []
    for i, (x, y) in enumerate(loader):
        if i >= max_batches:
            break
        x, y = x.to(device), y.to(device)
        _, loss = model(x, y)
        losses.append(loss.item())
    model.train()
    return sum(losses) / len(losses) if losses else 0

def train_phase(phase_config, phase_idx):
    """Train on a single curriculum phase."""
    global global_iter, best_val_loss
    
    print(f"\n{'='*60}")
    print(f"PHASE {phase_idx + 1}: {phase_config['name']}")
    print(f"{'='*60}\n")
    
    # Load dataset
    train_dataset = CurriculumDataset(
        phase_config['files'],
        block_size=train_config.block_size
    )
    
    if len(train_dataset) == 0:
        print('No data, skipping...')
        return
    
    train_loader = DataLoader(
        train_dataset,
        batch_size=train_config.batch_size,
        shuffle=True,
        num_workers=2,
        pin_memory=True
    )
    
    # Validation loader (if available)
    val_files = phase_config.get('val_files', phase_config['files'])
    val_dataset = CurriculumDataset(val_files, block_size=train_config.block_size)
    val_loader = DataLoader(val_dataset, batch_size=train_config.batch_size, shuffle=False)
    
    phase_lr = phase_config['lr']
    
    for epoch in range(phase_config['epochs']):
        epoch_losses = []
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{phase_config['epochs']}")
        
        for batch_idx, (x, y) in enumerate(pbar):
            # Update learning rate
            lr = get_lr(global_iter, phase_lr)
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr
            
            # Forward
            x, y = x.to(device), y.to(device)
            _, loss = model(x, y)
            loss = loss / train_config.gradient_accumulation_steps
            loss.backward()
            
            epoch_losses.append(loss.item() * train_config.gradient_accumulation_steps)
            
            # Gradient accumulation
            if (batch_idx + 1) % train_config.gradient_accumulation_steps == 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), train_config.grad_clip)
                optimizer.step()
                optimizer.zero_grad()
                global_iter += 1
                
                # Update progress bar
                avg_loss = sum(epoch_losses[-20:]) / min(len(epoch_losses), 20)
                pbar.set_postfix({'loss': f'{avg_loss:.4f}', 'lr': f'{lr:.2e}'})
        
        # End of epoch evaluation
        train_loss = sum(epoch_losses) / len(epoch_losses)
        val_loss = evaluate(model, val_loader)
        
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['phase'].append(phase_idx)
        history['iteration'].append(global_iter)
        
        print(f"Epoch {epoch+1}: train_loss={train_loss:.4f}, val_loss={val_loss:.4f}")
        
        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save({
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'phase': phase_idx,
                'epoch': epoch,
                'val_loss': val_loss,
                'global_iter': global_iter,
            }, CHECKPOINT_DIR / 'best.pt')
            print(f'  → New best model saved!')
        
        # Check if target reached
        if val_loss < phase_config['target_loss']:
            print(f"Target loss {phase_config['target_loss']} reached, advancing...")
            break
    
    # Save phase checkpoint
    torch.save({
        'model_state_dict': model.state_dict(),
        'phase': phase_idx,
        'val_loss': val_loss,
    }, CHECKPOINT_DIR / f'phase{phase_idx+1}_complete.pt')

print('Training function defined')

In [ ]:
# Run curriculum training!
print('\n' + '#'*60)
print('STARTING CURRICULUM TRAINING')
print('#'*60)

start_time = time.time()

for phase_idx, phase_config in enumerate(PHASES):
    train_phase(phase_config, phase_idx)

total_time = time.time() - start_time
print(f"\n{'#'*60}")
print(f'TRAINING COMPLETE')
print(f'Total time: {total_time/60:.1f} minutes')
print(f'Best validation loss: {best_val_loss:.4f}')
print(f"{'#'*60}")

---
## 4. Evaluation & Comparison

In [ ]:
# Plot training curves
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss curves
ax = axes[0]
ax.plot(history['train_loss'], label='Train', alpha=0.8)
ax.plot(history['val_loss'], label='Validation', alpha=0.8)
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')
ax.set_title('Training Loss by Epoch')
ax.legend()
ax.grid(True, alpha=0.3)

# Add phase boundaries
phase_changes = []
for i in range(1, len(history['phase'])):
    if history['phase'][i] != history['phase'][i-1]:
        phase_changes.append(i)
        ax.axvline(x=i, color='red', linestyle='--', alpha=0.5)

# Loss by phase
ax = axes[1]
phase_names = ['P1: Primitives', 'P2: Composition', 'P3: Algorithms', 'P4: Systems', 'P5: Language']
phase_final_losses = {}
for i, p in enumerate(history['phase']):
    if p not in phase_final_losses:
        phase_final_losses[p] = []
    phase_final_losses[p].append(history['val_loss'][i])

final_losses = [phase_final_losses.get(i, [0])[-1] for i in range(5)]
ax.bar(range(5), final_losses, color=['#3498db', '#2ecc71', '#f39c12', '#e74c3c', '#9b59b6'])
ax.set_xticks(range(5))
ax.set_xticklabels(phase_names, rotation=15)
ax.set_ylabel('Final Validation Loss')
ax.set_title('Final Loss by Phase')
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(CHECKPOINT_DIR / 'training_curves.png', dpi=150)
plt.show()

In [ ]:
# Test generation
def generate_text(prompt, max_tokens=200, temperature=0.8):
    """Generate text from prompt."""
    model.eval()
    
    # Encode
    tokens = list(prompt.encode('utf-8'))
    idx = torch.tensor(tokens, dtype=torch.long, device=device).unsqueeze(0)
    
    # Generate
    with torch.no_grad():
        output = model.generate(idx, max_new_tokens=max_tokens, temperature=temperature, top_k=50)
    
    # Decode
    return bytes(output[0].tolist()).decode('utf-8', errors='replace')

# Test prompts
prompts = [
    'The dragon emerged from the cave and',
    'def fibonacci(n):',
    'In 1945, the war ended when',
]

print('='*60)
print('GENERATION SAMPLES')
print('='*60)

for prompt in prompts:
    print(f"\nPrompt: {prompt}")
    print('-'*40)
    output = generate_text(prompt, max_tokens=150)
    print(output)
    print()

---
## 5. Save Results

In [ ]:
# Save training history
with open(CHECKPOINT_DIR / 'history.json', 'w') as f:
    json.dump(history, f, indent=2)

# Save config
config_dict = {
    'n_layer': config.n_layer,
    'n_head': config.n_head,
    'n_embd': config.n_embd,
    'vocab_size': config.vocab_size,
    'dropout': config.dropout,
    'mlp_internal_dim_multiplier': config.mlp_internal_dim_multiplier,
}
with open(CHECKPOINT_DIR / 'config.json', 'w') as f:
    json.dump(config_dict, f, indent=2)

print(f'Results saved to {CHECKPOINT_DIR}')
!ls -la {CHECKPOINT_DIR}

In [ ]:
# Copy to Google Drive (Colab only)
if IN_COLAB:
    !cp -r {CHECKPOINT_DIR} /content/drive/MyDrive/bdh_curriculum/
    print('Checkpoints copied to Google Drive')

---
## Summary

This notebook trained BDH through 5 curriculum phases:

1. **Primitives** - Simple functions (add, subtract, etc.)
2. **Composition** - Loops, recursion, control flow
3. **Algorithms** - Sorting, searching, data structures
4. **Systems** - OOP classes, design patterns
5. **Language** - WikiText-2 natural language

**Next Steps:**
- Compare with baseline (WikiText-2 only) training
- Evaluate on downstream tasks
- Try different phase orderings
- Experiment with phase mixing strategies